<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Инициализация" data-toc-modified-id="Инициализация-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Инициализация</a></span></li><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Календарь" data-toc-modified-id="Календарь-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Календарь</a></span></li><li><span><a href="#Цены-на-товары-каждую-неделю" data-toc-modified-id="Цены-на-товары-каждую-неделю-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Цены на товары каждую неделю</a></span></li><li><span><a href="#Количество-покупок-каждого-товара-за-каждый-день-исследумого-периода" data-toc-modified-id="Количество-покупок-каждого-товара-за-каждый-день-исследумого-периода-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Количество покупок каждого товара за каждый день исследумого периода</a></span></li><li><span><a href="#Представление-ответа" data-toc-modified-id="Представление-ответа-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Представление ответа</a></span></li></ul></li><li><span><a href="#Модель" data-toc-modified-id="Модель-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Модель</a></span></li></ul></div>

# M5 Forecasting - Accuracy
# Изотов Илья

## Инициализация

In [1]:
#!pip install numpy pandas
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./kaggle/input\calendar.csv
./kaggle/input\sales_train_validation.csv
./kaggle/input\sample_submission.csv
./kaggle/input\sell_prices.csv
./kaggle/input\m5-forecasting-accuracy\calendar.csv
./kaggle/input\m5-forecasting-accuracy\sales_train_validation.csv
./kaggle/input\m5-forecasting-accuracy\sample_submission.csv
./kaggle/input\m5-forecasting-accuracy\sell_prices.csv


In [2]:
# Подключение своего модуля для загрузки и анализа данных
import sys
sys.path.insert(0, "./../exploratory_analysis")
import expdataanalysis as eda

Загрузка данных

In [3]:
calendar = eda.read_file('./kaggle/input\calendar.csv')
sell_prices = eda.read_file('./kaggle/input\sell_prices.csv')
train_validation = eda.read_file('./kaggle/input\sales_train_validation.csv')
submission = eda.read_file('./kaggle/input/sample_submission.csv')

reading csv file...
reading csv file...
reading csv file...
reading csv file...


## Обзор данных

### Календарь

In [4]:
"""
date - дата
wm_yr_wk - номер недели
weekday - день недели
wday - номер дня недели
month - номер месяца
year - год
d - номер дня
event_name_1 - имя события, если оно происходило в этот день
event_type_1 - тип события, если оно происходило в этот день
event_name_2 - имя события 2, если оно происходило в этот день
event_type_2 - тип события 2, если оно происходило в этот день
snap_CA, snap_TX, snap_WI - (SNAP - программа помощи в питании) - показывает было ли в этот день по этому штату распределение пособия по питанию
"""
print(calendar.shape)
calendar.head(5)

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [5]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [6]:
# Спискок праздиников
calendar.event_name_1.unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [7]:
# Дни, когда два праздника 
calendar[~(calendar.event_type_2.isna())]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
85,2011-04-24,11113,Sunday,2,4,2011,d_86,OrthodoxEaster,Religious,Easter,Cultural,0,0,0
827,2013-05-05,11315,Sunday,2,5,2013,d_828,OrthodoxEaster,Religious,Cinco De Mayo,Cultural,1,1,1
1177,2014-04-20,11412,Sunday,2,4,2014,d_1178,Easter,Cultural,OrthodoxEaster,Religious,0,0,0
1233,2014-06-15,11420,Sunday,2,6,2014,d_1234,NBAFinalsEnd,Sporting,Father's day,Cultural,0,1,1
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


### Цены на товары каждую неделю 

In [8]:
"""
store_id - идентификатор магазина (включает наименование штата)
item_id - идентификатор продукта (включает категорию и подкатегорию товара)
wm_yr_wk - номер недели
sell_price - цена товара в данном магазине на данной неделе
"""
print(sell_prices.shape)
sell_prices.sample(5)

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
3320560,TX_1,FOODS_3_434,11621,2.88
3441856,TX_2,HOBBIES_1_146,11234,4.66
4910631,WI_1,HOBBIES_2_060,11431,0.97
5055479,WI_1,HOUSEHOLD_2_039,11435,9.27
3607145,TX_2,HOUSEHOLD_1_319,11416,1.87


### Количество покупок каждого товара за каждый день исследумого периода

In [9]:
"""
id - обозначает тоже самое, что в submission с той лишь разницей, что тут данные только для открытой части соревнования
item_id - идентификатор товара (FOODS_3_629 - 629 товар 3-й подкатегории категории Еда)
dept_id - идентификатор подкатегории
cat_id - идентификатор категории
store_id - идентификатор магазина
state_id - идентификатор штата
d_1..d_1913 - сведения о количестве товаров купленных по каждому товару за каждый день
"""
print(train_validation.shape)
train_validation.sample(5)

(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
4446,HOUSEHOLD_2_302_CA_2_validation,HOUSEHOLD_2_302,HOUSEHOLD_2,HOUSEHOLD,CA_2,CA,0,0,0,0,...,0,6,2,3,0,3,0,2,1,0
30008,FOODS_3_343_WI_3_validation,FOODS_3_343,FOODS_3,FOODS,WI_3,WI,1,0,0,0,...,0,1,2,0,0,0,0,0,0,0
4761,FOODS_1_103_CA_2_validation,FOODS_1_103,FOODS_1,FOODS,CA_2,CA,2,4,4,2,...,5,7,3,0,1,6,2,1,3,1
18382,HOBBIES_1_091_TX_3_validation,HOBBIES_1_091,HOBBIES_1,HOBBIES,TX_3,TX,0,0,0,0,...,1,1,1,0,4,0,0,0,0,2
16066,HOUSEHOLD_1_262_TX_2_validation,HOUSEHOLD_1_262,HOUSEHOLD_1,HOUSEHOLD,TX_2,TX,0,0,0,0,...,0,1,1,3,4,2,2,4,0,1


### Представление ответа

In [10]:
"""id - HOBBIES_2_004_WI_1 - говорит о том, что это 4 позиция товара 2-й подкатегории товаров категории "Хобби" в 1-м магазине штата WI
validation - значит, что данный id необходимо использовать для предсказаний открытой части соревнования - прогноз с 1914 по 1941 день
evaluation - значит, что данный id необходимо использовать для предсказаний закрытой части соревнования - прогноз с 1942 по 1969 день
F1 - F28 - 28 дней по которым необходимо предсказать количество покупок данного товара """
print(submission.shape)
submission.sample(5)

(60980, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
32972,FOODS_3_258_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41745,FOODS_2_282_CA_4_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46133,HOBBIES_1_407_TX_2_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45937,HOBBIES_1_209_TX_2_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37687,HOUSEHOLD_2_003_CA_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Модель

In [11]:
# импортируем библиотеки для работы со временем, очистки от лишних ссылок в памяти, биб
from  datetime import datetime, timedelta
import gc
import lightgbm as lgb

# задаем переменные
pd.options.display.max_columns = 50
# соответствие имен столбцов их типам данных
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

FIRST_DAY = 1 # Если хотите загрузить все данные, то установите '1' -->  Но при этом высокий риск, что не хватит памяти !
h = 28 # количество дней, на которые создается предсказание
max_lags = 730 # основываясь на каком количестве дней мы будем строить предсказание 
tr_last = 1913 # последний день для тренировки
fday = datetime(2016,4, 25) # дата первого дня для предсказания
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [12]:
# собираем все таблицы в одну
def create_dt(is_train = True, nrows = None, first_day = 1200):
    # таблица цен. Кодируем категориальные признанки в числа типа int
    prices = pd.read_csv("./kaggle/input\sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
    # таблица календарь. Кодируем категориальные признанки в числа типа int
    cal = pd.read_csv("./kaggle/input\calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    # вычисляем день, начиная с которого будем использовать данные
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)] # выбираем числовые данные
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id'] # выбираем категориальные данные
    dtype = {numcol:"float32" for numcol in numcols} # задаем тип данных для числовых столбцов
    dtype.update({col: "category" for col in catcols if col != "id"})
    # скачиваем данные о покупках
    dt = pd.read_csv("./kaggle/input\sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    # кодируем категориальные признаки
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    # если имеем дело с тренировочным набором данных, то задаем пустые дни для предсказания
    if not is_train:
        for day in range(tr_last+1, tr_last+ 2*h +1):
            dt[f"d_{day}"] = np.nan
    # 
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [13]:
def create_fea(dt):
    lags = [5, 7, 14, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [5, 7, 14, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [14]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

Wall time: 32.9 s


(46027957, 22)

In [15]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
1,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
2,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0.46
4,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0.46


In [16]:
df = eda.optimize_mem_usage(df)

In [17]:
%%time

create_fea(df)
df.shape

Wall time: 7min 51s


(46027957, 45)

In [18]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_5,lag_7,lag_14,lag_28,rmean_5_5,rmean_7_5,rmean_14_5,rmean_28_5,rmean_5_7,rmean_7_7,rmean_14_7,rmean_28_7,rmean_5_14,rmean_7_14,rmean_14_14,rmean_28_14,rmean_5_28,rmean_7_28,rmean_14_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,29
1,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,30
2,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,31
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,1
4,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,2


In [19]:
df.dropna(inplace = True)
df.shape

(44351007, 45)

In [20]:
df = eda.optimize_mem_usage(df)

In [21]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [22]:
train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
fake_valid_inds = np.random.choice(len(X_train), 1000000)
fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
                             free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [23]:
params = {
        "objective" : "poisson",
        #"objective" : "regression",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.150,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 4200,
#     'device' : 'gpu'
}

In [24]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=100) 

C:\Users\Ilya\anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Ilya\anaconda\lib\site-packages\lightgbm\basic.py:1382: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\Ilya\anaconda\lib\site-packages\lightgbm\basic.py:1117: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\Ilya\anaconda\lib\site-packages\lightgbm\basic.py:929: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[100]	valid_0's rmse: 3.17181
[200]	valid_0's rmse: 2.45732
[300]	valid_0's rmse: 2.41321
[400]	valid_0's rmse: 2.37839
[500]	valid_0's rmse: 2.34877
[600]	valid_0's rmse: 2.32877
[700]	valid_0's rmse: 2.31641
[800]	valid_0's rmse: 2.29811
[900]	valid_0's rmse: 2.2886
[1000]	valid_0's rmse: 2.27693
[1100]	valid_0's rmse: 2.26365
[1200]	valid_0's rmse: 2.25531
[1300]	valid_0's rmse: 2.24527
[1400]	valid_0's rmse: 2.23756
[1500]	valid_0's rmse: 2.23136
[1600]	valid_0's rmse: 2.22641
[1700]	valid_0's rmse: 2.22106
[1800]	valid_0's rmse: 2.21648
[1900]	valid_0's rmse: 2.21172
[2000]	valid_0's rmse: 2.2079
[2100]	valid_0's rmse: 2.20265
[2200]	valid_0's rmse: 2.19951
[2300]	valid_0's rmse: 2.19558
[2400]	valid_0's rmse: 2.19205
[2500]	valid_0's rmse: 2.18867
[2600]	valid_0's rmse: 2.18547
[2700]	valid_0's rmse: 2.18274
[2800]	valid_0's rmse: 2.17938
[2900]	valid_0's rmse: 2.17614
[3000]	valid_0's rmse: 2.17283
[3100]	valid_0's rmse: 2.17005
[3200]	valid_0's rmse: 2.16804
[3300]	valid_0's rm

In [25]:
%%time

te = create_dt(False)
te.shape

Wall time: 16.7 s


(22998241, 22)

In [26]:
df = eda.optimize_mem_usage(df)

In [27]:
%%time

for i in range(0, 28):
    day = fday + timedelta(days=i)
    print(i, day)
    tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
    create_fea(tst)
    tst = tst.loc[tst.date == day , train_cols]
    te.loc[te.date == day, "sales"] = 1.02*m_lgb.predict(tst) # magic multiplier by kyakovlev

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
Wall time: 2h 58min 48s


In [28]:
%%time

te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
                                                                      "id"].str.replace("validation$", "evaluation")
te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][[f"F{i}" for i in range(1,29)]].reset_index()
te_sub.fillna(0., inplace = True)
te_sub.to_csv("submission.csv",index=False)
te_sub.shape

Wall time: 6.57 s
Parser   : 189 ms


(60980, 29)

In [29]:
te_sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,FOODS_1_001_CA_1_validation,0.913571,0.779885,0.808077,0.784062,0.959566,1.154251,1.159661,1.056923,0.981981,0.917495,0.906498,1.104612,1.284642,1.059523,0.947204,0.856203,0.894193,0.904055,0.998203,1.257240,1.153200,0.922052,0.819652,0.824684,0.828592,0.994934,1.182890,1.118106
2,FOODS_1_001_CA_2_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,FOODS_1_001_CA_2_validation,0.851170,0.836483,0.953065,1.008892,1.188032,1.505652,1.636197,1.043646,1.108154,1.002583,1.113018,1.118262,1.523714,1.244362,1.032336,0.961155,1.009070,1.062627,1.207013,1.618970,1.609665,1.135278,1.063750,1.085231,1.001101,1.187224,1.577262,1.456487
4,FOODS_1_001_CA_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,FOODS_1_001_CA_3_validation,0.961153,0.945979,0.863973,0.878282,0.876648,1.201112,1.241614,0.996720,1.037722,0.966199,1.057531,1.078490,1.365748,1.137632,1.057787,1.077372,1.030761,1.029889,1.128593,1.491002,1.443052,1.124531,1.044562,1.023161,1.053922,1.090367,1.196606,1.187918
6,FOODS_1_001_CA_4_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,FOODS_1_001_CA_4_validation,0.358794,0.332311,0.379022,0.382485,0.444300,0.463635,0.514300,0.377602,0.365012,0.379286,0.375871,0.401297,0.415313,0.380752,0.377480,0.369024,0.389671,0.379690,0.417107,0.470682,0.466890,0.384271,0.367376,0.383897,0.381998,0.419012,0.449005,0.428693
8,FOODS_1_001_TX_1_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,FOODS_1_001_TX_1_validation,0.158230,0.108106,0.106293,0.106546,0.130375,0.303708,0.327377,0.331654,0.374351,0.357049,0.361017,0.402771,0.464290,0.377714,0.345576,0.335430,0.351141,0.324715,0.369877,0.372454,0.363356,0.299112,0.292101,0.308013,0.302825,0.330508,0.373946,0.352538
